In [2]:
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl import function as fn
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh

In [ ]:

import networkx as nx
import matplotlib.pyplot as plt
import random as rand
import numpy as np

#set gpu if available
if th.cuda.is_available():
    print("GPU is available")
    device = th.device("cuda")
else:
    print("GPU not available, CPU used")
    device = th.device("cpu")

In [154]:
#operation for neigbors
class NodeApplyModule(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        if self.activation is not None:
            h = self.activation(h)
        return {'h' : h}
    
#gcn layer in network
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        
        g.ndata['h'] = feature
        g.pull(g.nodes())
        g.apply_nodes(self.apply_mod)
        
        return g.ndata.pop('h')
    
#network
class GCN_LL(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCN_LL, self).__init__()
        self.gcn1 = GCN(in_feats, 100, F.relu)
        self.gcn2 = GCN(100, 30, F.relu)
        self.gcn3 = GCN(30, out_feats, th.tanh)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        x = self.gcn2(g, x)
        
        return self.gcn3(g, x)


In [192]:
#loss function

"""Here is an implementation for your similarity_matrix using only matrix operations. It can run on the GPU and is going to be significantly faster than your previous implementation.
https://discuss.pytorch.org/t/build-your-own-loss-function-in-pytorch/235/6
"""

"""
Issue and to-do:
    Similarity matrix results in nan
"""

def similarity_matrix(mat):
    # get the product x * y
    # here, y = x.t()
    r = th.mm(mat, mat.t())
    if th.sum(th.isnan(r)) > 0:
        print(th.sum(th.isnan(mat)))
        print(th.sum(th.isnan(r)))
    # get the diagonal elements
    diag = r.diag().unsqueeze(0)
    diag = diag.expand_as(r)
    # compute the distance matrix
    D = diag + diag.t() - 2*r
    return D.sqrt()

def same_label(y):
    s = y.size(0)
    y_expand = y.unsqueeze(0).expand(s, s)
    Y = y_expand.eq(y_expand.t())
    return Y

def my_loss(output, labels):
    """
    if nodes with the same label: x^2
    if nodes with different label: 1/(x+0.01) #so the result is never infinite
    """
    sim = similarity_matrix(output)
    temp = same_label(labels)
    same_l = (temp * sim) / (output.size(1))
    same_l_inv = ((temp*(-1) + 1) * sim) / (output.size(1))
    
    loss = (th.sum(same_l)**2) + (1/(th.sum(same_l_inv)+0.001))
    
    if loss != loss:
        print(th.sum(th.isnan(sim)))
        print(loss)
    
    return loss


In [4]:
#load dataset
data = citegrh.load_cora()
ds_features = th.FloatTensor(data.features) #convert to pytorch data type #######
ds_labels = th.LongTensor(data.labels)
ds_train = th.ByteTensor(data.train_mask)
ds_test = th.ByteTensor(data.test_mask)
ds_g = data.graph

# add self loop for the sum of festures
ds_g.remove_edges_from(nx.selfloop_edges(ds_g))
ds_g = DGLGraph(ds_g)
ds_g.add_edges(ds_g.nodes(), ds_g.nodes())
ds_g.ndata['label'] = ds_labels #used to filter and train, not needed for prediction

m_func = fn.copy_src(src='h', out='m')
m_reduce_func = fn.sum(msg='m', out='h')

In [156]:
########### Create Model ############

#constant parameters
DIST_VEC_SIZE = 10

model = GCN_LL(ds_features.size()[1], DIST_VEC_SIZE)
## opt = th.optim.Adam(model.parameters(), lr=1e-3); only run once

In [161]:
#training

r = list(range(2, int(len(ds_g)*0.1))) #10 percent of all the nodes

model.train()
for epoch in range(10):
    rand.shuffle(r)
    
    error = []
    acc = [0.0] #accuracy
    for count in r:
        sub_graph = ds_g.subgraph(ds_g.nodes()[:count])
        sub_graph.copy_from_parent()
        sub_graph.register_message_func(m_func)
        sub_graph.register_reduce_func(m_reduce_func)
        
        feats = ds_features[:count]
        labs = ds_labels[:count]
        
        out = model(sub_graph, feats)
        loss = my_loss(out, labs)
        error.append(loss.item())
        
        opt.zero_grad()
        loss.backward()
        opt.step()
    
    print(f"Epoch {epoch}: \n\tavg error = {np.mean(error)} \n\tlast error = {error[-1]} \n\tavg accuracy = {np.mean(acc)} \n\tlast accuracy = {acc[-1]} ")
    
    if(epoch%44 == 0 and DIST_VEC_SIZE == 2):
        pass

Epoch 0: 
	avg error = nan 
	last error = 46.05598831176758 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 1: 
	avg error = nan 
	last error = 1941.4166259765625 
	avg accuracy = 0.0 
	last accuracy = 0.0 
Epoch 2: 
	avg error = nan 
	last error = 2990.134521484375 
	avg accuracy = 0.0 
	last accuracy = 0.0 


KeyboardInterrupt: 

1433

In [193]:
r = list(range(2, 200))
rand.shuffle(r)
model.train()
for i in r:
    sub_graph = ds_g.subgraph(ds_g.nodes()[:i])
    sub_graph.copy_from_parent()
    sub_graph.register_message_func(m_func)
    sub_graph.register_reduce_func(m_reduce_func)

    feats = ds_features[:i]
    labs = ds_labels[:i]

    out = model(sub_graph, feats)

    loss = my_loss(out, labs)
    #print(loss)
    print("----------------------")
    
    opt.zero_grad()
    loss.backward()
    opt.step()



----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
tensor(1)
tensor(nan, grad_fn=<AddBackward0>)
----------------------
tensor(1)
tensor(nan, grad_fn=<AddBackward0>)
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
tensor(1)
tensor(nan, grad_fn=<AddBackward0>)
----------------------
----------------------
----------------------
-----------